In [54]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from typing import List
from sklearn.model_selection import train_test_split

In [49]:
# Quick test to see how many duplicates are in these csv's
df_1 = pd.read_csv("../data/dev.csv")
df_2 = pd.read_csv("../data/train.csv")
duplicate_count = 0
for i in range(len(df_1)):
    if (df_2[df_2.eq(df_1.iloc[i, 0]).any(axis=1)].empty == False):
        df_2_index = int(df_2[df_2.eq(df_1.iloc[i, 0]).any(axis=1)].iloc[0].name)
        if (df_2.iloc[df_2_index,1] == df_1.iloc[i, 1]):
            duplicate_count += 1
    
    
    if (df_2[df_2.eq(df_1.iloc[i, 1]).any(axis=1)].empty == False):
        df_2_index = int(df_2[df_2.eq(df_1.iloc[i, 1]).any(axis=1)].iloc[0].name)
        if (df_2.iloc[df_2_index,0] == df_1.iloc[i, 0]):
            duplicate_count += 1

print(duplicate_count)

1


In [56]:
def split_data(
    FIRST_TEXTS: List[str],
    SECOND_TEXTS: List[str],
    LABELS: List[int],
    test_split: float = 0.1
    ) -> [List[str], List[str], List[str], List[str], List[str], List[str]]:
    """
    Splits a set of first and second texts and their labels into a training/test split
    args:
        FIRST_TEXTS,
        SECOND_TEXTS,
        LABELS,
        test_split: float - the amount of test data points to extract from the existing set of data
    returns:
        2 sets of data
    """

    # Init test lists and amended lists
    TEST_FIRST_TEXTS, TEST_SECOND_TEXTS, TEST_LABELS = [],[],[]
    TRAIN_FIRST_TEXTS, TRAIN_SECOND_TEXTS, TRAIN_LABELS = [],[],[]

    TRAIN_FIRST_TEXTS, TEST_FIRST_TEXTS, TRAIN_SECOND_TEXTS, \
    TEST_SECOND_TEXTS, TRAIN_LABELS, TEST_LABELS = train_test_split(
        FIRST_TEXTS, 
        SECOND_TEXTS, 
        LABELS, 
        test_size=test_split,
        random_state=1234
    )

    return TRAIN_FIRST_TEXTS, TRAIN_SECOND_TEXTS, TRAIN_LABELS, TEST_FIRST_TEXTS, TEST_SECOND_TEXTS, TEST_LABELS

In [29]:
def prepareData(csv_filepath: str, verbose: bool = False) -> [List[str], List[str], List[int]]:
    """
    Prepares the data from csv files
    args:
        csv_filepath: str - the filepath of the csv file to load our authorship verification data from
        verbose: bool
    returns:
        FIRST_TEXTS: List[str] - list of strings in the first column of csv 
        SECOND_TEXTS: List[str] - list of strings in the second column of csv 
        LABELS: List[int] - one hot encoded labels for whether the first and second text are from the same author
    """
    FIRST_TEXTS, SECOND_TEXTS, LABELS = [],[],[]

    # Load the dataframe
    df = pd.read_csv(csv_filepath)
    # Iterate through and add to our first and second texts and labels
    # I was initially worried about this being slow but it manages to do this on the train dataset very fast
    for i in range(len(df)):
        FIRST_TEXTS.append(df.iloc[i, 0])
        SECOND_TEXTS.append(df.iloc[i, 1])
        LABELS.append(df.iloc[i, 2])
    
    # Ensure that the data is valid
    assert len(FIRST_TEXTS) == len(SECOND_TEXTS) == len(LABELS)
    if verbose: print("Prepared", len(df), "data points.")

    return FIRST_TEXTS, SECOND_TEXTS, LABELS

In [57]:
TRAIN_FIRST_TEXTS, TRAIN_SECOND_TEXTS, TRAIN_LABELS  = prepareData("../data/train.csv", True)
DEV_FIRST_TEXTS, DEV_SECOND_TEXTS, DEV_LABELS = prepareData("../data/dev.csv", True)

TRAIN_FIRST_TEXTS, TRAIN_SECOND_TEXTS, TRAIN_LABELS, \
TEST_FIRST_TEXTS, TEST_SECOND_TEXTS, TEST_LABELS = split_data(TRAIN_FIRST_TEXTS, TRAIN_SECOND_TEXTS, TRAIN_LABELS)

print("Training data size: ", len(TRAIN_FIRST_TEXTS),
     "\nDev data size: ", len(DEV_FIRST_TEXTS), 
     "\nTest data size: ", len(TEST_FIRST_TEXTS))

Prepared 30000 data points.
Prepared 6000 data points.
Training data size:  27000 
Dev data size:  6000 
Test data size:  3000


In [ ]:
class Dataset():
    """
    For a given dataset, whether that's the training, dev, or 
    test dataset, this class handles most of it's setup and functions.

    We will do all tokenization, feature extraction within this class and it will be what we pass to the model etc.

    Variables:
        FIRST_TEXTS - a list of the first texts,
        SECOND_TEXTS - a list of the second texts,
        LABELS - the binary classification (0 or 1) of whether this is from the same author,
        FIRST_TOKENS - a list of the first texts, tokenized,
        SECOND_TOKENS - a list of the first texts, tokenized,
    """

    def __init__(self, FIRST_TEXTS, SECOND_TEXTS, LABELS):
        self.FIRST_TEXTS = FIRST_TEXTS
        self.SECOND_TEXTS = SECOND_TEXTS
        self.LABELS = LABELS
    
    def ExtractFeatures(self):



## USE THIS: LightningDataModule


In [9]:
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")

input_ids = tokenizer("The <extra_id_0> walks in <extra_id_1> park", return_tensors="pt").input_ids
print(input_ids)
print(type(input_ids))


tensor([[   37, 32099, 10681,    16, 32098,  2447,     1]])
<class 'torch.Tensor'>
